# 서울시 주요 공원 현황

In [12]:
import numpy as np 
import pandas as pd
import seaborn as sns
import matplotlib as mpl 
import matplotlib.pyplot as plt 
import folium
import json
%matplotlib inline
# 한글폰트 사용
mpl.rc('font', family='Malgun Gothic')
mpl.rc('axes', unicode_minus=False)

In [13]:
df = pd.read_csv('./data/서울시 주요 공원현황.csv', sep=',', encoding='euc-kr')
df.head(2)

,공원번호,공원명,공원개요,면적,개원일,주요시설,주요식물,안내도,오시는길,이용시참고사항,이미지,지역,공원주소,관리부서,전화번호,X좌표(GRS80TM),Y좌표(GRS80TM),X좌표(WGS84),Y좌표(WGS84),바로가기
0,1,남산도시자연공원,남산공원은 도심에 위치하여 서울시민에게 맑은 공기를 제공하는 자연휴식처이며 산책 꽃...,2896887㎡ 임 야 : 2454140㎡ 녹지대 및 기타시설 : 442747㎡,1968.9.10,기반시설 : 광장 45950㎡ 도로 108530㎡ 산책로 6.7㎞ (북측:3.7㎞/...,소나무 단풍 아카시아 상수리나무 등 191종 2881870주,http://parks.seoul.go.kr/upload/seditorMulti/2...,명동역(남산케이블카 와룡묘 서울애니메이션센터 방면) 도보 명동역 3번출구 → 퍼시픽...,남산공원 차량통행 금지안내 2005년 5월 1일부터 남산공원에 일반승용차 택시 통행...,http://parks.seoul.go.kr/file/info/view.do?fId...,중구,서울특별시 중구 삼일대로 231(예장동),서울특별시중부푸른도시사업소,02-3783-5900,198364.107,450395.554,126.990377,37.550140,http://parks.seoul.go.kr/template/sub/namsan.do
1,3,길동생태공원,길동생태공원은 생물의 서식처를 제공하고 종다양성을 증진시키며 자연생태계의 생물들을 ...,80683㎡,1999.5.20,탐방객안내소 야외전시대 관찰대 목재데크 조류관찰대 길동생태문화센터,소나무 보리수 등 64종 31800주 산국 부들 등 138종 192800본,http://parks.seoul.go.kr/template/common/img/p...,NaN,매주 월요일은 휴관입니다 사전예약 후 입장하실 수 있습니다. ▶ 인터넷 예약 바로가...,http://parks.seoul.go.kr/file/info/view.do?fId...,강동구,서울특별시 강동구 천호대로 1291(길동생태공원),동부공원녹지사업소 길동생태공원,02-489-2770,213554.120,448852.675,127.154779,37.540394,http://parks.seoul.go.kr/template/sub/gildong.do


In [14]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 132 entries, 0 to 131
Data columns (total 20 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   공원번호          132 non-null    int64  
 1   공원명           132 non-null    object 
 2   공원개요          131 non-null    object 
 3   면적            129 non-null    object 
 4   개원일           122 non-null    object 
 5   주요시설          124 non-null    object 
 6   주요식물          88 non-null     object 
 7   안내도           102 non-null    object 
 8   오시는길          115 non-null    object 
 9   이용시참고사항       120 non-null    object 
 10  이미지           132 non-null    object 
 11  지역            131 non-null    object 
 12  공원주소          132 non-null    object 
 13  관리부서          132 non-null    object 
 14  전화번호          132 non-null    object 
 15  X좌표(GRS80TM)  122 non-null    float64
 16  Y좌표(GRS80TM)  122 non-null    float64
 17  X좌표(WGS84)    131 non-null    float64
 18  Y좌표(WGS84)    131 non-null    

In [15]:
del df['공원개요']
del df['개원일']
del df['주요시설']
del df['주요식물']
del df['안내도']
del df['오시는길']
del df['이용시참고사항']
del df['이미지']
del df['관리부서']
del df['전화번호']
del df['X좌표(GRS80TM)']
del df['Y좌표(GRS80TM)']
del df['바로가기']
df.head()

,공원번호,공원명,면적,지역,공원주소,X좌표(WGS84),Y좌표(WGS84)
0,1,남산도시자연공원,2896887㎡ 임 야 : 2454140㎡ 녹지대 및 기타시설 : 442747㎡,중구,서울특별시 중구 삼일대로 231(예장동),126.990377,37.550140
1,3,길동생태공원,80683㎡,강동구,서울특별시 강동구 천호대로 1291(길동생태공원),127.154779,37.540394
2,4,서울대공원,9132690m²,과천시,경기도 과천시 대공원광장로 102,127.019847,37.426449
3,5,서울숲,480994㎡,성동구,서울특별시 성동구 뚝섬로 273 (성수동1가),127.041798,37.543072
4,6,월드컵공원,2284085㎡,마포구,서울특별시 마포구 하늘공원로 84(월드컵공원),126.878907,37.571805


In [17]:
del df['공원번호']
df.head()

,공원명,면적,지역,공원주소,X좌표(WGS84),Y좌표(WGS84)
0,남산도시자연공원,2896887㎡ 임 야 : 2454140㎡ 녹지대 및 기타시설 : 442747㎡,중구,서울특별시 중구 삼일대로 231(예장동),126.990377,37.550140
1,길동생태공원,80683㎡,강동구,서울특별시 강동구 천호대로 1291(길동생태공원),127.154779,37.540394
2,서울대공원,9132690m²,과천시,경기도 과천시 대공원광장로 102,127.019847,37.426449
3,서울숲,480994㎡,성동구,서울특별시 성동구 뚝섬로 273 (성수동1가),127.041798,37.543072
4,월드컵공원,2284085㎡,마포구,서울특별시 마포구 하늘공원로 84(월드컵공원),126.878907,37.571805


In [28]:
df.fillna('0', inplace=True)
tmp_list = df['면적'].tolist()
area_list = []
for a in tmp_list:
    area = a.split(maxsplit=1)[0]
    if area == '총':
        area = a.split(maxsplit=1)[1]
    else:
        area = a.split(maxsplit=1)[0]
    area_list.append(area)
print(area_list)

['2896887㎡', '80683㎡', '9132690m²', '480994㎡', '2284085㎡', '8948.1㎡', '휴양', '6456㎥', '80309㎡', '1980.4㎡', '5197.7㎡', '11467㎡', '229539㎡', '15179.7', '16734.60㎥', '61544㎡', '424106㎡', '560552㎡', '26696.8㎥', '75900㎡', '113021.7m2', '201779㎥', '258991㎡', '10420819.08㎡', '171294.1㎥', '15000㎡', '297926㎡', '411972㎡', '98470㎡', '2038741.3㎡', '603715.2㎡', '108120㎡', '29619.3㎥', '29843.8㎡', '126026㎡', '135002.7㎡', '384026㎡', '180000㎥', '416031㎡', '160720㎡', '2277252㎡', '19599.70㎥', '138596㎡', '185295㎥', '254648㎡', '29816.5㎡', '762642㎡', '664905㎡', '99599.9㎡', '5802㎡', '16500㎥ ', '585652㎥', '649709.6㎡', '72132㎡', '66027.8㎡', '265582㎡', '2088704.9㎡', '112070㎡', '993724㎡', '219167㎡', '1349556.4㎡', '1511198㎡', '1097565㎡', '328018㎡', '1447122㎡', '207790㎡', '1018689㎡', '954553㎡', '1667848㎡', '13224㎡', '130345㎡', '5332422㎡', '5107964㎡', '47832㎡', '213552㎡', '168099.87㎡', '359435.2㎡', '36660㎡', '380735㎡', '1807420.9㎡', '359342.8㎡', '632733㎡', '328018㎡', '6691885.3㎡', '285757㎡', '110407㎡', '29682.2㎡', '

In [30]:
area_list[6] = '0'
print(area_list)

['2896887㎡', '80683㎡', '9132690m²', '480994㎡', '2284085㎡', '8948.1㎡', '0', '6456㎥', '80309㎡', '1980.4㎡', '5197.7㎡', '11467㎡', '229539㎡', '15179.7', '16734.60㎥', '61544㎡', '424106㎡', '560552㎡', '26696.8㎥', '75900㎡', '113021.7m2', '201779㎥', '258991㎡', '10420819.08㎡', '171294.1㎥', '15000㎡', '297926㎡', '411972㎡', '98470㎡', '2038741.3㎡', '603715.2㎡', '108120㎡', '29619.3㎥', '29843.8㎡', '126026㎡', '135002.7㎡', '384026㎡', '180000㎥', '416031㎡', '160720㎡', '2277252㎡', '19599.70㎥', '138596㎡', '185295㎥', '254648㎡', '29816.5㎡', '762642㎡', '664905㎡', '99599.9㎡', '5802㎡', '16500㎥ ', '585652㎥', '649709.6㎡', '72132㎡', '66027.8㎡', '265582㎡', '2088704.9㎡', '112070㎡', '993724㎡', '219167㎡', '1349556.4㎡', '1511198㎡', '1097565㎡', '328018㎡', '1447122㎡', '207790㎡', '1018689㎡', '954553㎡', '1667848㎡', '13224㎡', '130345㎡', '5332422㎡', '5107964㎡', '47832㎡', '213552㎡', '168099.87㎡', '359435.2㎡', '36660㎡', '380735㎡', '1807420.9㎡', '359342.8㎡', '632733㎡', '328018㎡', '6691885.3㎡', '285757㎡', '110407㎡', '29682.2㎡', '1

In [33]:
import re
tmp = []
for b in area_list:
    area = re.sub('㎡|m²|m2|㎥', '', b)
    tmp.append(area)
print(tmp)

['2896887', '80683', '9132690', '480994', '2284085', '8948.1', '0', '6456', '80309', '1980.4', '5197.7', '11467', '229539', '15179.7', '16734.60', '61544', '424106', '560552', '26696.8', '75900', '113021.7', '201779', '258991', '10420819.08', '171294.1', '15000', '297926', '411972', '98470', '2038741.3', '603715.2', '108120', '29619.3', '29843.8', '126026', '135002.7', '384026', '180000', '416031', '160720', '2277252', '19599.70', '138596', '185295', '254648', '29816.5', '762642', '664905', '99599.9', '5802', '16500 ', '585652', '649709.6', '72132', '66027.8', '265582', '2088704.9', '112070', '993724', '219167', '1349556.4', '1511198', '1097565', '328018', '1447122', '207790', '1018689', '954553', '1667848', '13224', '130345', '5332422', '5107964', '47832', '213552', '168099.87', '359435.2', '36660', '380735', '1807420.9', '359342.8', '632733', '328018', '6691885.3', '285757', '110407', '29682.2', '137759', '917190', '51470', '662543', '217946', '179666', '34955', '18083', '4835', '196

In [34]:
df = pd.DataFrame({
    '구별': df['지역'],
    '공원명': df['공원명'],
    '공원주소': df['공원주소'],
    '면적': tmp,
    '위도': df['Y좌표(WGS84)'],
    '경도': df['X좌표(WGS84)']
})
df.head()

,구별,공원명,공원주소,면적,위도,경도
0,중구,남산도시자연공원,서울특별시 중구 삼일대로 231(예장동),2896887,37.5501,126.99
1,강동구,길동생태공원,서울특별시 강동구 천호대로 1291(길동생태공원),80683,37.5404,127.155
2,과천시,서울대공원,경기도 과천시 대공원광장로 102,9132690,37.4264,127.02
3,성동구,서울숲,서울특별시 성동구 뚝섬로 273 (성수동1가),480994,37.5431,127.042
4,마포구,월드컵공원,서울특별시 마포구 하늘공원로 84(월드컵공원),2284085,37.5718,126.879


In [35]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 132 entries, 0 to 131
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   구별      132 non-null    object
 1   공원명     132 non-null    object
 2   공원주소    132 non-null    object
 3   면적      132 non-null    object
 4   위도      132 non-null    object
 5   경도      132 non-null    object
dtypes: object(6)
memory usage: 6.3+ KB


In [36]:
df = df.astype({
    '면적': float,
    '위도': float,
    '경도': float
    })
df.head()

,구별,공원명,공원주소,면적,위도,경도
0,중구,남산도시자연공원,서울특별시 중구 삼일대로 231(예장동),2896887.0,37.550140,126.990377
1,강동구,길동생태공원,서울특별시 강동구 천호대로 1291(길동생태공원),80683.0,37.540394,127.154779
2,과천시,서울대공원,경기도 과천시 대공원광장로 102,9132690.0,37.426449,127.019847
3,성동구,서울숲,서울특별시 성동구 뚝섬로 273 (성수동1가),480994.0,37.543072,127.041798
4,마포구,월드컵공원,서울특별시 마포구 하늘공원로 84(월드컵공원),2284085.0,37.571805,126.878907


### 위도, 경도 정보가 맞는지 다시 가져오기 - 카카오맵 이용

In [38]:
import requests
from urllib.parse import urlparse
import pandas as pd 
key_fd = open('kakaomapkey.txt', mode='r')
kmap_key = key_fd.read(100)
key_fd.close()

In [41]:
lat_list = []; lng_list = []
for i in df.index:
    print(i)
    url = 'https://dapi.kakao.com/v2/local/search/address.json?query=' + df['공원주소'][i]
    result = requests.get(urlparse(url).geturl(), headers={"Authorization":"KakaoAK " + kmap_key}).json()
    lat = float(result['documents'][0]['y'])
    lng = float(result['documents'][0]['x'])

    lat_list.append(lat); lng_list.append(lng)

0
1
2
3
4
5
6
7
8


IndexError: list index out of range

In [42]:
df.head(10)

,구별,공원명,공원주소,면적,위도,경도
0,중구,남산도시자연공원,서울특별시 중구 삼일대로 231(예장동),2896887.0,37.550140,126.990377
1,강동구,길동생태공원,서울특별시 강동구 천호대로 1291(길동생태공원),80683.0,37.540394,127.154779
2,과천시,서울대공원,경기도 과천시 대공원광장로 102,9132690.0,37.426449,127.019847
3,성동구,서울숲,서울특별시 성동구 뚝섬로 273 (성수동1가),480994.0,37.543072,127.041798
4,마포구,월드컵공원,서울특별시 마포구 하늘공원로 84(월드컵공원),2284085.0,37.571805,126.878907
5,종로구,광화문시민열린마당,서울특별시 종로구 세종로 76-2,8948.1,37.575045,126.978023
6,은평구,진관근린공원(구파발폭포),서울특별시 은평구 진관동 393,0.0,37.637024,126.919216
7,영등포구,중마루근린공원,서울특별시 영등포구 영등포동2가 222,6456.0,37.519832,126.911267
8,성동구,응봉공원,서울특별시 성동구 금호동1가 산1-72,80309.0,37.557228,127.021765
9,강서구,매화근린공원,서울특별시 강서구 등촌동 630-14,1980.4,37.557583,126.861150
